In [1]:
import sys, os
sys.path.append("/Users/miiya/python_code-set/lib")
import numpy as np

show_pyplot = False
show_plotly = False

if (show_pyplot):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import axes3d
if (show_plotly):
    import plotly.offline as offline
    import plotly.graph_objs as go
    offline.init_notebook_mode(connected=True)

In [2]:
ifile = "/Users/miiya/data/tmp_wave_spherical/NBSwave_A1_Y40.3d"
d  = np.loadtxt(ifile)
Nd = int(len(d[:,0])**(1/3)+0.0001)
d[0,3] = 0.0
print(Nd)
print( d)

96
[[  0.          0.          0.          0.          0.       ]
 [  0.          0.          1.          0.4936659   0.       ]
 [  0.          0.          2.          0.4936659   0.       ]
 ..., 
 [ 95.         95.         93.          0.3718522   0.       ]
 [ 95.         95.         94.          0.2997257   0.       ]
 [ 95.         95.         95.          0.235079    0.       ]]


In [3]:
Ns = 2
sd = np.array([[[d[iz + Nd*(iy + Nd*ix),:] 
                 for iz in range(0,Nd,Ns)] for iy in range(0,Nd,Ns)] for ix in range(0,Nd,Ns)])
Nsd = len(sd[:,0,0,0])
sd  = sd.reshape((Nsd**3,5))
print(Nsd)
print( sd)

48
[[  0.          0.          0.          0.          0.       ]
 [  0.          0.          2.          0.4936659   0.       ]
 [  0.          0.          4.          0.4936659   0.       ]
 ..., 
 [ 94.         94.         90.          0.3718522   0.       ]
 [ 94.         94.         92.          0.2997257   0.       ]
 [ 94.         94.         94.          0.235079    0.       ]]


In [4]:
sdc = sd
for ix in range(Nsd):
    for iy in range(Nsd):
        for iz in range(Nsd):
            idx = iz + Nsd*(iy + Nsd*ix)
            for i in range(3):
                if (sdc[idx,i] >= Nd//2):
                    sdc[idx,i] = sd[idx,i]-Nd
print(sdc)

[[ 0.         0.         0.         0.         0.       ]
 [ 0.         0.         2.         0.4936659  0.       ]
 [ 0.         0.         4.         0.4936659  0.       ]
 ..., 
 [-2.        -2.        -6.         0.3718522  0.       ]
 [-2.        -2.        -4.         0.2997257  0.       ]
 [-2.        -2.        -2.         0.235079   0.       ]]


In [5]:
if (show_pyplot):
    %matplotlib notebook
    fig = plt.figure()
    axe = fig.gca(projection='3d')
    sca = axe.scatter3D(sdc[:,0],sdc[:,1],sdc[:,2], c=sdc[:,3], s=2, cmap=plt.cm.viridis)
    axe.set_xlabel('x')
    axe.set_ylabel('y')
    axe.set_zlabel('z')
    plt.colorbar(sca)
    plt.show()

In [6]:
if (show_plotly):
    offline.iplot(dict(data=[go.Scatter3d(x=sdc[:,0],y=sdc[:,1],z=sdc[:,2], mode='markers',
                                          marker=dict(color=sdc[:,3], size=sdc[:,3]**3*100,
                                                      colorscale='Jet', opacity=0.4, showscale=True))],
                                          #projection=dict(x=dict(show=True),
                                          #                y=dict(show=True),
                                          #                z=dict(show=True)))],
                       layout=dict(margin=dict(l=0,r=0,b=0,t=0), autosize=False, width=800, height=800)),
                  filename='test')

In [7]:
if (show_plotly):
    data = [dict(x=sdc[:Nsd**2,1], y=sdc[:Nsd**2,2], visible=False, mode='markers',
                 marker=dict(color=sdc.reshape((Nsd,Nsd,Nsd,5))[:,:,iz,3].reshape((Nsd**2)),
                             size =sdc.reshape((Nsd,Nsd,Nsd,5))[:,:,iz,3].reshape((Nsd**2))*30,
                             colorscale='Jet', showscale=True),
                 name='z = '+str(sdc[iz,2]))
            for iz in range(Nsd)]
    
    data[0]['visible'] = True
    
    steps = []
    for i in range(len(data)):
        step = dict(method='restyle', args=['visible', [False]*len(data)])
        step['args'][1][i] = True
        steps.append(step)
    
    sliders = [dict(active=6, currentvalue={"prefix": "Frequency: "}, pad={"t": 50}, steps=steps)]
    
    offline.iplot(dict(data=data,
                       layout=dict(sliders=sliders, margin=dict(l=0,r=0,b=0,t=0),
                                   autosize=False, width=800, height=800)),
                  filename='test')

In [8]:
if (show_plotly):
    offline.iplot(dict(data=[go.Surface(x=sdc[:,0].reshape((Nsd,Nsd,Nsd))[:Nsd//2,0,:Nsd//2],
                                        y=sdc[:,2].reshape((Nsd,Nsd,Nsd))[:Nsd//2,0,:Nsd//2],
                                        z=sdc[:,3].reshape((Nsd,Nsd,Nsd))[:Nsd//2,0,:Nsd//2],
                                        contours=dict(x=dict(show=True,project=dict(x=True)),
                                                      y=dict(show=True,project=dict(y=True)),
                                                      z=dict(show=True,project=dict(z=True))))],
                       layout=dict(autosize=False, width=800, height=800, margin=dict(l=0,r=0,b=0,t=0))),
                  filename='test')